# llm 기본틀 만들기
- OpenAI Key 로딩, ChatOpenAI() 객체 생성

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv('../.env', override=True)

api_key = os.getenv('OPENAI_API_KEY')
# print(api_key)

# 모델 객체 생성

In [2]:
# gpt model 객체 생성
# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000254718F30E0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025471825970>, root_client=<openai.OpenAI object at 0x00000254708176B0>, root_async_client=<openai.AsyncOpenAI object at 0x00000254718A1B80>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 시스템 프롬프트 작성

- https://github.com/f/awesome-chatgpt-prompts
검색 : financial
```
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- "Can you tell us what future stock market looks like based upon current conditions ?".
```

In [4]:
system_prompt = """
Want assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".
"""
user_prompt = "{company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요."

# 프롬프트 객체 생성

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])
prompt

ChatPromptTemplate(input_variables=['company'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nWant assistance provided by qualified individuals enabled with experience on understanding charts using technical analysis tools while interpreting macroeconomic environment prevailing across world consequently assisting customers acquire long term advantages requires clear verdicts therefore seeking same through informed predictions written down precisely! First statement contains following content- “Can you tell us what future stock market looks like based upon current conditions ?".\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company'], input_types={}, partial_variables={}, template='{company}에 주식을 투자해도 될까요? 마크다운 형식의 투자보고서를 한글로 작성해 주세요.'), additional_kwargs={})])

# OutPut 파서 객체 생성

In [6]:
output_parser = StrOutputParser()
output_parser

StrOutputParser()

# LCEL로 LLM 요청 및 응답받기

- chain 구성 : LCEL(LangChain Expression Language)

In [18]:
# from stock_info import Stock 인식이 안될 경우 아래 코드 실행
import sys
import os

# 상위 디렉터리(LANGCHAIN_PJT)를 경로에 추가
sys.path.append(os.path.abspath(".."))


In [19]:
from stock_info import Stock

# 회사 이름
company = "MicroSoft"
symbol = "MSFT"

stock = Stock(symbol)
stock


In [ ]:
chain = prompt | llm | output_parser

response = chain.invoke({"company":company})

In [8]:
print(response)

### Microsoft 주식에 대한 투자 분석

#### 기업 개요:
Microsoft (NASDAQ: MSFT)는 전세계적으로 가장 큰 소프트웨어 기업 중 하나로 자리 잡고 있습니다. 회사는 클라우드 컴퓨팅, 인공지능 및 기타 혁신 기술 분야에서 선도적인 지위를 유지하고 있으며 글로벌 시장에서 큰 영향력을 발휘하고 있습니다.

#### 기술적 분석:
Microsoft 주식은 최근 상승세를 보이고 있습니다. 이동평균선 및 상대강도지수 (RSI) 분석을 통해 주식의 추세가 강세를 유지하고 있음을 확인할 수 있습니다. 기술적 지표들은 현재 Microsoft 주식에 대한 매수 신호를 보여주고 있으며 향후 상승 가능성이 높은 것으로 보입니다.

#### 기본적 분석:
Microsoft는 안정적인 재무 상태를 유지하고 있으며 지속적으로 수익성을 향상시키고 있습니다. 회사의 클라우드 사업과 오피스 제품군은 안정적인 성장을 보여주고 있으며 향후에도 성장 가능성이 크다고 판단됩니다.

#### 투자 결론:
Microsoft는 기술적, 기본적 분석 측면에서 모두 투자 가치가 있는 기업으로 평가됩니다. 현재 주식 시장의 전반적인 상황과 Microsoft의 신제품 출시 등 다양한 요인들을 고려할 때, Microsoft 주식에 대한 투자는 장기적으로 안정적인 수익을 기대할 수 있는 좋은 기회가 될 수 있습니다.

#### 투자 시 주의사항:
그러나 주식 시장은 변동성이 높은 특성을 가지고 있기 때문에 투자할 때에는 자신의 투자 목표와 위험 감수 수준을 신중히 고려해야 합니다. 전문가와 상담하거나 투자 전략을 확립하는 데 도움을 받는 것이 중요합니다.

이 보고서는 Microsoft 주식에 대한 일반적인 분석을 제공한 것이며 투자 결정을 내리실 때에는 개별적인 상황을 고려해야 합니다.
